# Xarray with Dask

Analysis of Gridded Ensemble Precipitation Estimates over Australia

- Authors: NCI Virtual Research Environment Team
- Keywords: Dask, xarray
- Creation Date: 2021-May
- Lineage/Reference: This tutorial is referenced to [Pangeo example](https://github.com/pangeo-data/pangeo-example-notebooks/blob/master/cm26.ipynb).
---


In this notebook we work with several ensemble members of precipitation data. For non-climate scientists ensemble roughly means that we take several predictions and compare them. 

The analysis we do below is quite simple but the problem is a good illustration of a common task in the atmospheric sciences.  We show how dask can be used to significantly accelerate typical `xarray` workflows and really highlights the synergies between these two tools.  We will:

* Open a dataset
* Extract average seasonal precipitation
* Extract a time series of annual maximum precipitation events over a region

The following material uses Coupled Model Intercomparison Project (CMIP6) collections. The CMIP6 terms of use are found [here](https://pcmdi.llnl.gov/CMIP6/TermsOfUse/TermsOfUse6-1.html). For more information on the collection, please visit the [NCI Data Catalogue]( https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f6600_2266_8675_3563).

---

### Setup

Create a client:

In [5]:
# If you run this notebook on your local computer or NCI's VDI instance, you can create cluster
from dask.distributed import Client
client = Client()
print(client)
print(client.dashboard_link)

<Client: 'tcp://127.0.0.1:55496' processes=4 threads=4, memory=8.00 GiB>
http://127.0.0.1:8787/status


Starting the Dask Client will provide a dashboard which is useful to gain insight into the computation. The link to the dashboard will become visible when you create the Client. We recommend having the Client open on one side of your screen and your notebook open on the other side, which will be useful for learning purposes.

**Amazingly** xarray can automatically use dask with minimal outside intervention if it detects there is a cluster available.

In [6]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [7]:
!ls ./CMIP6-ACCESS/r1/
!ls ./CMIP6-ACCESS/r2/
!ls ./CMIP6-ACCESS/r3/

pr_day_ACCESS-CM2_historical_r1i1p1f1_gn_18500101-18991231.nc
pr_day_ACCESS-CM2_historical_r1i1p1f1_gn_19000101-19491231.nc
pr_day_ACCESS-CM2_historical_r1i1p1f1_gn_19500101-19991231.nc
pr_day_ACCESS-CM2_historical_r1i1p1f1_gn_20000101-20141231.nc
pr_day_ACCESS-CM2_historical_r2i1p1f1_gn_18500101-18991231.nc
pr_day_ACCESS-CM2_historical_r2i1p1f1_gn_19000101-19491231.nc
pr_day_ACCESS-CM2_historical_r2i1p1f1_gn_19500101-19991231.nc
pr_day_ACCESS-CM2_historical_r2i1p1f1_gn_20000101-20141231.nc
pr_day_ACCESS-CM2_historical_r3i1p1f1_gn_18500101-18991231.nc
pr_day_ACCESS-CM2_historical_r3i1p1f1_gn_19000101-19491231.nc
pr_day_ACCESS-CM2_historical_r3i1p1f1_gn_19500101-19991231.nc


### Open Dataset

Here we load the historical precipitation data of the ACCESS-CM2 model within the CMIP6 archive into an `xarray` dataset. 

If you are taking this course in conjunction ith our `xarray` course some of this will be very familiar. Note the use of the `parallel=True` kwarg.



In [8]:
EnsNum=3
DS=[]
for i in range(EnsNum): #loop through ensemble members
    #concatenate data on time coordinate
    data=xr.open_mfdataset('./CMIP6-ACCESS/r'+str(i+1)+'/pr_day_ACCESS-CM2_historical_r*_gn_*.nc',combine='nested', concat_dim='time', parallel=True )
    DS.append(data)

The dataset has dimensions of time, latitude, longitude, and ensemble members.

We need to join our datasets together. 

In [9]:
ds=xr.concat([DS[i] for i in range(EnsNum)],'ensemble')

/Users/hugo/opt/anaconda3/envs/nci/lib/python3.9/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


### Intra-ensemble range

We calculate the intra-ensemble range for all the mean daily temperature in this dataset.  This gives us a sense of uncertainty.

In [10]:
pr_mean = ds['pr'].mean(dim='time')
spread = (pr_mean.max(dim='ensemble')
        - pr_mean.min(dim='ensemble'))
spread

<xarray.DataArray 'pr' (lat: 144, lon: 192)>
dask.array<sub, shape=(144, 192), dtype=float32, chunksize=(144, 192), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * lon      (lon) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 357.2 359.1

### Calling compute
As with just about any dask things we have done so far, the expressions above didn't actually compute anything but rather just build the task graph. To do the computations, we call the `compute` or `persist` methods. Compute returns a the underlying representation (numpy array) while persist keeps the dask representation.

In [ ]:
spread.compute()

In [ ]:
spread = spread.persist()

We can also trigger a flush off disk using `load` but we won't here.

In [ ]:
#spread.load()

Lets make a figure of our intra-ensemble range.

We will keep it simple in this notebook and just use colourmaps. See our `xarray` tutorial for good looking holographic projections.

In [ ]:
spread.plot(robust=True, figsize=(10, 6))
plt.title('Intra-ensemble range in mean annual precipitation')

### Average seasonal precipitation

We can compute a crude estimate of average seasonal precipitation variables in our dataset. Here, we'll look at the first 2 ensemble members and make some maps of the seasonal total precipitation in each ensemble member.

First we resample our data to be **quartely** with the year starting in **March**, indicated by the `time=QS-MAR` tag. We then sum over the time axis `.sum(time)`

In [ ]:
da_pr = ds['pr'].resample(time='QS-MAR').sum('time')

We then slice along the `ensemble` axis and compute a seasonal mean. 

In [ ]:

seasonal_pr = da_pr.isel(ensemble=slice(0, 2)).groupby('time.season').mean('time').compute()


We can now sort out the seasons properly, which our `QS-MAR` time slice was an ugly short hand for. 

In [ ]:
# properly sort the seasons
seasonal_pr = seasonal_pr.sel(season=['DJF', 'MAM','JJA', 'SON'])
seasonal_pr

#### Average seasonal precipitation totals 
Now lets find our plot our average seasonal precipitation totals.

In [ ]:
seasonal_pr.plot.pcolormesh(col='season', row='ensemble', cmap='Blues', robust=True)
plt.show()

### Extract a time series of annual maximum precipitation events over a region

In the previous two examples, we've mostly reduced the time and/or ensemble dimension. Here, we'll do a reduction operation on the spatial dimension to look at a time series of extreme precipitation events near Sydney, AU (33.51° S, 151.12° E).

In [ ]:
buf = 0.25  # look at Sydney +/- 0.25 deg

ds_syd = ds.sel(lon=slice(151.12-buf, 151.12+buf), lat=slice(-33.51-buf, 33.51+buf))

We resample to be annually.

In [ ]:
pcp_ann_max = ds_syd['pr'].resample(time='AS').max('time')

We find the maximum value over our `lat` and `lon` range.

In [ ]:
pcp_ann_max_ts = pcp_ann_max.max(('lat', 'lon')).compute()

### Timeseries of maximum precipitation near Sydney, AU.
Now lets make a plot of our annual precipitation!

In [ ]:
ax = pcp_ann_max_ts.transpose().to_pandas().plot(title='Maximum precipitation near Sydney, AU', legend=False)

## Challenge

Choose another place on the globe and compute its maximum precipitation! 


In [ ]:
# choose a different longitude and latitiude  and range to examine!

### Close the client

Before moving on to the next exercise, make sure to close your client or stop this kernel.

In [ ]:
client.close()

## Reference

This example is modified from https://github.com/pangeo-data/pangeo-example-notebooks/blob/master/xarray-data.ipynb

## Conclusion

Working with `xarray` is fun and easy, especially for gridded data. If you did this course in tandem with the `xarray` course hopefully this has taken your `xarray` skills to the next level. 

How easy was integrating `Dask` and `xarray`! These two tools have fantastic synergy and should be used together wherever possible.

**Jump over to [Notebook 5](./dask_gpu_distributed_05.ipynb) now.** 